In [1]:
# Load environment variables from a .env file
from dotenv import load_dotenv
import os

load_dotenv(override=True)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    api_key=GOOGLE_API_KEY,
)

In [19]:
from pydantic import BaseModel
from typing import Literal

class Router(BaseModel):
    """Decide the next agent to route to"""
    next_agent: Literal["weather", "flights", "__end__"]

supervisor_model = llm.with_structured_output(Router)

In [23]:
from langgraph.graph import MessagesState
from langgraph.types import Command
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

# Supervisor Agent

def supervisor(state: MessagesState) -> Command:
    """The central supervisor agent that routes to the correct agent or ends the conversation."""
    print("==[ Supervisor ]==")

    sys_prompt = """You are a supervisor agent routing tasks to a specialist agent.
    Based on the user's request, choose the appropriate agent only from the followin list.

    Available agent:
    - Weather: For questions about weather forecasts.
    - Flights: For questions about flight information.

    If the user says thank you or the conversation is over, choose '__end__'.
    """

    user_prompt = state["messages"][-1].content

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", sys_prompt),
            ("user", user_prompt)
        ]
    )

    if isinstance(state['messages'][-1], HumanMessage):
        response = supervisor_model.invoke(prompt.format_messages())
        print(f"Supervisor routing to: {response.next_agent}")
        return Command(goto=response.next_agent.lower())
    else:
        return Command(goto='__end__')
    

# Specialist Agents

def weather_agent(state: MessagesState) -> Command:
    """A specialist agents for handling weather related queries"""
    print("*==[ Weather Agent ]==*")
    sys_prompt = "You are a weather forecaster. Provide a concise mock weather forecast for the location mentioned in the user's message."
    prompt = ChatPromptTemplate.from_messages([
        ("system", sys_prompt),
        ("user", state["messages"][-1].content)
    ])
    response = llm.invoke(prompt.format_messages())
    print(f"Response: {response.content}")
    # Return to the supervisor after run
    return Command(
        goto="supervisor",
        update={"messages": [response]}
    )

def flights_agent(state: MessagesState) -> Command:
    """A specialist agents for handling flight status-related queries"""
    print("*==[ Flight Agent ]==*")
    sys_prompt = "You are a flight information assistant. Provide some mock flight details for the destination in the user's message. Respond with short concise information."
    prompt = ChatPromptTemplate.from_messages([
        ("system", sys_prompt),
        ("user", state["messages"][-1].content)
    ])
    response = llm.invoke(prompt.format_messages())
    print(f"Response: {response.content}")
    # Return to the supervisor after run
    return Command(
        goto="supervisor",
        update={"messages": [response]}
    )

In [24]:
from langgraph.graph import StateGraph, START, END

# Build the graph
workflow = StateGraph(MessagesState)

workflow.add_node("supervisor", supervisor)
workflow.add_node("weather", weather_agent)
workflow.add_node("flights", flights_agent)

workflow.add_edge(START, "supervisor")

# Compile graph
graph = workflow.compile()

In [25]:
# TEST
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    # graph.stream() invokes the graph and streams the result
    events = graph.stream({"messages": [HumanMessage(content=user_input)]})
    for event in events:
        # Only print AI's response here
        if "messages" in event:
            event["messages"][-1].pretty_print()

==[ Supervisor ]==
Supervisor routing to: weather
*==[ Weather Agent ]==*
Supervisor routing to: weather
*==[ Weather Agent ]==*
Response: Good morning, Maldives! Get ready for a beautiful day across the islands. Expect **sunny skies** with a gentle breeze. Temperatures will be comfortably warm, topping out around **30 degrees Celsius (86 degrees Fahrenheit)**. There's a **very low chance of a brief, isolated shower** this afternoon, but overall, it's a perfect day for enjoying the beaches and turquoise waters.
==[ Supervisor ]==
Response: Good morning, Maldives! Get ready for a beautiful day across the islands. Expect **sunny skies** with a gentle breeze. Temperatures will be comfortably warm, topping out around **30 degrees Celsius (86 degrees Fahrenheit)**. There's a **very low chance of a brief, isolated shower** this afternoon, but overall, it's a perfect day for enjoying the beaches and turquoise waters.
==[ Supervisor ]==
==[ Supervisor ]==
==[ Supervisor ]==
Supervisor routing 